In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
from ast import literal_eval
from helper_functions.get_website_sentences_from_mongo import get_website_sentences_from_mongo
from helper_functions.process_sentences import process_sentences
from helper_functions.get_NER_format import get_NER_format, NER_maker
from helper_functions.get_healthcare_api_response import get_healthcare_api_response
from helper_functions.get_healthcare_api_labels import get_healthcare_api_labels

In [ ]:
df = get_website_sentences_from_mongo()

In [15]:
df = process_sentences(df)

In [23]:
df.head(5)

,drug_token,drug,bert_classification,sentence,fda_approved,date_epoch,date,source,mean,company_name,url,drugs
0,austerity,austerity,not_drug,government-debt crisis countermeasures * 2000s...,0,1608768000,"December 24, 2020",NaN,0.000000,endpts,https://endpts.com/acordas-ron-cohen-brings-th...,"[(austerity, not_drug)]"
1,urothelial,urothelial,not_drug,Keytruda first gained approval in a bladder ca...,0,1608681600,"December 23, 2020",NaN,0.000000,endpts,https://endpts.com/fda-ushers-mercks-star-canc...,"[(urothelial, not_drug)]"
2,remdesivir,remdesivir,drug,remdesivir is also authorized to treat pediatr...,0,1605916800,"November 21, 2020",NaN,1.065624,medpagetoday,https://www.medpagetoday.com/infectiousdisease...,"[(remdesivir, drug)]"
3,rp_3500,RP-3500,drug,ventures-founded biotech plans to start testin...,0,1609200000,"December 29, 2020",NaN,1.060803,fiercebiotech,https://www.fiercebiotech.com/biotech/versant-...,"[(RP-3500, drug)]"
4,vonapanitase,vonapanitase,drug,Proteon is testing vonapanitase in peripheral ...,0,1609545600,"January 02, 2021",NaN,1.064360,fiercebiotech,https://www.fiercebiotech.com/biotech/proteon-...,"[(vonapanitase, drug)]"


In [25]:
res, failed = get_NER_format(df.head(1000))

In [72]:
import copy
df = copy.deepcopy(res)

In [73]:
label_map = {
        'Drug': ['drug','not_drug'],
    
        'O': ['O', 'value', 'disease', 'unit',
           'organization', 'health_care_activity', 'finding',
           'chemical_substance', 'body_part_or_organ_or_organ_component',
           'gene', 'physiologic_function', 'cell_and_cell_components',
           'drug_frequency', 'drug_route', 'drug_form', 'drug_unit',
           'drug_strength', 'concept', 'drug_duration', 'date',
           'sign_or_symptom', 'medical_device', 'drug_dosage', 'tissue']}

In [74]:
relabel_dic = {}
for elem in label_map: 
    for subelem in label_map[elem]:
        relabel_dic[subelem] = elem

In [75]:
label_new = [relabel_dic[n] if n in relabel_dic else n for n in df.label]
df['new_label'] = label_new

In [76]:
import json 
d1 = df.to_json(orient = 'records')
data = json.loads(d1)

In [77]:
new_data = [] 
for n in data: 
    if n['new_label'] in ['Drug', 'Target', 'Indication', 'Company']: 
        count1 = 0
        for n1 in n['word'].split(): 
            if count1 == 0: 
                lab = 'B-{}'.format(n['new_label'])
            else: 
                lab = 'I-{}'.format(n['new_label'])
            new_data.append({
                'word': n1, 
                'label': lab
            })
            count1+=1
    elif n['new_label'] == 'O': 
        for n2 in n['word'].split(): 
            new_data.append({
                'word': n2, 
                'label': 'O'
            })
    else: 
        new_data.append({
                'word': None, 
                'label': None
            })

In [86]:
pd.DataFrame(new_data).to_csv('final.tsv', sep = '\t', header = 0, index = False)